<a href="https://colab.research.google.com/github/jihansyahmhd/Final_Project/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installasi Library

In [ ]:
import importlib.metadata

desired_version = "1.26.4"

try:
    installed_version = importlib.metadata.version("numpy")
    if installed_version == desired_version:
        print(f"NumPy {desired_version} is already installed.")
    else:
        print(f"Installing NumPy {desired_version} (current: {installed_version})...")
        !pip install numpy=={desired_version} --prefer-binary
        import os
        os._exit(00)  # Restart runtime for changes to take effect
except importlib.metadata.PackageNotFoundError:
    print(f"NumPy is not installed. Installing {desired_version}...")
    !pip install numpy=={desired_version} --prefer-binary
    import os
    os._exit(00)

Installing NumPy 1.26.4 (current: 2.0.2)...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 55.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
!pip install uv
# !uv pip install langchain chromadb transformers torch accelerate nvidia-pyindex nvidia-pycl nemo-toolkit[all] cudf-cu11 cuml-cu11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 96.5 MB/s eta 0:00:00


In [ ]:
# yes | pip uninstall torch torchvision torchaudio numpy transformers -y
#
# # Install compatible NumPy and PyTorch (CUDA 11.7 for T4)
!uv pip install --index-url https://download.pytorch.org/whl/cu121 torch torchvision torchaudio
#
# # Install Transformers, LangChain, ChromaDB, PyPDF2
!uv pip install transformers==4.33.0 langchain chromadb PyPDF2
#
# # Install NVIDIA NeMo from GitHub (Hydra & Lightning included)
BRANCH='main'
!uv pip install "git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]"
#
# # Install system utilities for audio/PDF
!apt-get update && apt-get install -y sox libsndfile1 ffmpeg
!pip install hydra-core omegaconf lightning text-unidecode wget

Streaming output truncated to the last 5000 lines.
nvidia-cublas-cu12       ------------------------------ 56.02 MiB/391.57 MiB
nvidia-cudnn-cu12        ------------------------------ 55.82 MiB/633.96 MiB
⠋ Preparing packages... (6/14)
nvidia-curand-cu12       ------------------------------ 31.10 MiB/53.85 MiB
nvidia-cufft-cu12        ------------------------------ 58.82 MiB/116.00 MiB
nvidia-cusolver-cu12     ------------------------------ 55.27 MiB/118.41 MiB
nvidia-cusparse-cu12     ------------------------------ 56.30 MiB/186.88 MiB
triton                   ------------------------------ 57.23 MiB/199.80 MiB
nvidia-cublas-cu12       ------------------------------ 56.02 MiB/391.57 MiB
nvidia-cudnn-cu12        ------------------------------ 56.44 MiB/633.96 MiB
⠋ Preparing packages... (6/14)
nvidia-curand-cu12       ------------------------------ 31.10 MiB/53.85 MiB
nvidia-cufft-cu12        ------------------------------ 59.14 MiB/116.00 MiB
nvidia-cusolver-cu12     -----------------

In [ ]:
!uv pip install -U langchain-community

In [ ]:
import nemo
print(nemo.__file__)

In [ ]:
!uv pip install sentence-transformers
!pip install langchain-nvidia-ai-endpoints


In [ ]:
!pip install gradio

In [ ]:
!pip install chromadb

# Build Chatbot

In [ ]:
import os
import re
import html
from typing import Dict, List, Tuple, Optional
import torch
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain.memory import ConversationBufferMemory
import gradio as gr


# ========== CONFIG ==========
JOURNAL_FOLDER = '/content/content/journals'
PERSIST_DIR = '/content/content/chroma_db'
MODEL_NAME = 'mistralai/mixtral-8x7b-instruct-v0.1'
GPU_DEVICE = 0

os.environ["NVIDIA_API_KEY"] = "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"

journal_content_map: Dict[str, str] = {}
embeddings = NVIDIAEmbeddings(model="nvidia/llama-3.2-nv-embedqa-1b-v2")

# ========== CLEANING ==========
def clean_text(text: str, max_length: int = 512) -> str:
    if not isinstance(text, str):
        return ""
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = html.unescape(text).strip()
    return text[:max_length]

# ========== LOAD & SPLIT ==========
def load_and_clean_pdfs(folder_path: str) -> List[dict]:
    all_docs = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.pdf'):
            try:
                loader = PyPDFLoader(os.path.join(folder_path, filename))
                docs = loader.load()
                if filename.lower() == "11-article text-51-1-10-20200802.pdf":  # File target
                  print(f"Jumlah halaman: {len(docs)}")
                  print(f"Contoh teks halaman 1: {docs[0].page_content[:200]}...")
                content = " ".join([d.page_content for d in docs if d.page_content])
                journal_content_map[filename] = content[:1500]

                splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=300)
                chunks = splitter.split_documents(docs)

                for chunk in chunks:
                    cleaned = clean_text(chunk.page_content, 1500)
                    if cleaned:
                        chunk.page_content = cleaned
                        all_docs.append(chunk)
            except Exception as e:
                print(f"❌ Failed loading {filename}: {e}")
    return all_docs

# ========== EMBEDDING ==========
def init_vector_store(docs: List[dict], persist_directory: str) -> Chroma:
    for doc in docs:
      doc.metadata["source"] = os.path.basename(doc.metadata["source"])
    return Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        persist_directory=persist_directory,
        collection_name='journals'
    )

# ========== QA CHAIN ==========
def init_qa_chain(vectordb: Chroma, model_name: str) -> ConversationalRetrievalChain:

    llm = ChatNVIDIA(model=model_name)
    retriever = vectordb.as_retriever(search_kwargs={'k': 4})

    # Inisialisasi memory
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,
        output_key="answer"
    )

    return ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        return_source_documents=True
    )

# ========== SUMMARIZATION ==========
def summarize_journal(filename: str) -> str:
    content = journal_content_map.get(filename, "")
    if not content:
        return "❌ Jurnal tidak ditemukan"

    llm = ChatNVIDIA(model=MODEL_NAME)
    prompt = PromptTemplate(
        input_variables=["judul", "content"],
        template="""
Anda adalah pakar riset akademik. Ringkas jurnal ini dengan struktur:
1. Judul: {judul}
2. abstrak
3. Latar Belakang
4. Metodologi
5. Hasil Utama
6. Kesimpulan

Isi:
{content}
"""
    )

    formatted = prompt.format(
        judul=os.path.splitext(filename)[0],
        content=content[:4000]
    )

    try:
        result = llm.invoke(formatted)
        return result.content
    except Exception as e:
        return f"❌ Ringkasan gagal: {e}"

PAGE_SIZE = 10
last_search_results: List[Tuple] = []
last_search_query: str = ""
last_page_index: int = 0
# ========== SEARCH ==========
def get_journal_titles_with_scores(query: str, vectordb: Chroma, page: int = 0) -> List[Tuple[str, str, float]]:
    global last_search_results, last_search_query, last_page_index

    # Hybrid search: semantic + keyword
    if query != last_search_query or page == 0:
        # Semantic search
        semantic_results = vectordb.similarity_search_with_score(query, k=50)

        # Keyword search manual
        keyword_matches = []
        for filename, content in journal_content_map.items():
            if query.lower() in content.lower():
                # Cari chunk yang mengandung keyword
                for doc in [d for d in docs if filename in d.metadata.get("source", "")]:
                    if query.lower() in doc.page_content.lower():
                        keyword_matches.append((doc, 1.0))  # Skor tinggi

        combined = semantic_results + keyword_matches
        unique_files = {}
        for doc, score in combined:
            source = doc.metadata.get("source", "")
            filename = os.path.basename(source)
            if filename not in unique_files or score < unique_files[filename][1]:
                unique_files[filename] = (doc, score)

        sorted_results = sorted(unique_files.items(), key=lambda x: x[1][1])
        last_search_results = [
            (os.path.splitext(fname)[0], fname, score)
            for fname, (doc, score) in sorted_results
        ]
        last_search_query = query
        last_page_index = 0

    start = page * PAGE_SIZE
    end = start + PAGE_SIZE
    return last_search_results[start:end]

def display_search_page_text(
    results_with_scores: List[Tuple],
    page: int,
    page_size: int
) -> str:
    """
    Display a single page of (Document, score) results.
    For each chunk, show:
      - index
      - base filename
      - similarity score
      - a short excerpt (first 200 chars of that chunk)
    """
    start_idx = page * page_size
    end_idx = start_idx + page_size
    page_results = results_with_scores[start_idx:end_idx]

    if not page_results:
        return "❌ Tidak ada hasil pada halaman ini."

    lines = [f"📄 Hasil pencarian halaman {page + 1}:\n"]
    for idx, (doc, score) in enumerate(page_results, start=1 + start_idx):
        # Ambil nama file dasar
        full_source = doc.metadata.get("source", "unknown")
        base_filename = os.path.splitext(os.path.basename(full_source))[0]

        # Ambil potongan (excerpt) 200 karakter pertama
        excerpt = doc.page_content[:200].replace("\n", " ").strip()
        lines.append(f"{idx}. [{base_filename}] (score: {score:.4f})")
        lines.append(f"    → {excerpt}...\n")

    # Jika masih ada halaman berikutnya, berikan petunjuk "next"
    if end_idx < len(results_with_scores):
        lines.append("Gunakan 'next' untuk melihat halaman berikutnya jika ada.")

    return "\n".join(lines)

def display_search_page(results_with_scores, page: int, page_size: int):
    start_idx = page * page_size
    end_idx = start_idx + page_size
    page_results = results_with_scores[start_idx:end_idx]

    # To avoid duplicates in the list of filenames
    unique_files = set()
    journal_list = []

    for doc, score in page_results:
        filename = os.path.basename(doc.metadata.get("source", "unknown"))
        base_filename = os.path.splitext(filename)[0]
        if base_filename not in unique_files:
            unique_files.add(base_filename)
            journal_list.append((base_filename, score))

    if not journal_list:
        print("❌ Tidak ada hasil pada halaman ini.")
        return

    print(f"\n📄 Hasil pencarian halaman {page + 1}:")
    for i, (title, score) in enumerate(journal_list, start=1):
        print(f"{i}. {title} (score: {score:.4f})")
    print("Gunakan 'next' untuk melihat halaman berikutnya jika ada.")

# # ========== CHAT LOOP ==========
# def chat_loop(qa_chain: RetrievalQA, vectordb: Chroma):
#     print("\n--- Journal Assistant (type 'exit' to quit) ---")
#     print("🟢 Gunakan:\n - search [topik]\n - next\n - summary [judul]\n - pertanyaan langsung\n")

#     last_search_query = None
#     last_search_results = []
#     current_page = 0
#     page_size = 10

#     while True:
#         user_input = input("You: ").strip()

#         if user_input.lower() in ('exit', 'quit'):
#             break

#         elif user_input.startswith("search "):
#             last_search_query = user_input[7:].strip()
#             current_page = 0
#             # fetch results with scores (max 100 for example)
#             last_search_results = vectordb.similarity_search_with_score(last_search_query, k=100)
#             last_search_results.sort(key=lambda x: x[1], reverse=True)

#             if not last_search_results:
#                 print("❌ Tidak ditemukan")
#                 continue

#             # Display first page
#             display_search_page(last_search_results, current_page, page_size)

#         elif user_input.lower() == "next":
#             if not last_search_query:
#                 print("❌ Tidak ada pencarian sebelumnya. Gunakan 'search [topik]' terlebih dahulu.")
#                 continue

#             current_page += 1
#             max_pages = (len(last_search_results) + page_size - 1) // page_size
#             if current_page >= max_pages:
#                 print("❌ Tidak ada halaman selanjutnya.")
#                 current_page = max_pages - 1
#             else:
#                 display_search_page(last_search_results, current_page, page_size)

#         elif user_input.startswith("summary "):
#             title = user_input[8:].strip()
#             matches = [f for f in journal_content_map if title.lower() in f.lower()]
#             if not matches:
#                 print("❌ Tidak ada jurnal dengan judul itu.")
#             elif len(matches) == 1:
#                 print(summarize_journal(matches[0]))
#             else:
#                 print("📚 Beberapa kandidat:")
#                 for i, f in enumerate(matches):
#                     print(f"{i+1}. {f}")
#                 idx = int(input("Pilih nomor: ")) - 1
#                 print(summarize_journal(matches[idx]))

#         else:
#             result = qa_chain(user_input)
#             print(f"\nBot: {result['result']}")
#             if result['source_documents']:
#                 shown = set()
#                 refs_found = False
#                 for doc in result['source_documents']:
#                     source = doc.metadata.get('source', '')
#                     if not source:
#                         continue
#                     base = os.path.splitext(os.path.basename(source))[0]
#                     if base not in shown:
#                         if not refs_found:
#                             # print("\n📚 Referensi:")
#                             refs_found = True
#                         # print(f"- {base}")
#                         shown.add(base)
#                 if not refs_found:
#                     print("\n📚 Tidak ada referensi yang relevan ditemukan.")
#             else:
#                 print("\n📚 Tidak ada referensi yang relevan ditemukan.")

# # ========== MAIN ==========
# if __name__ == '__main__':
#     docs = load_and_clean_pdfs(JOURNAL_FOLDER)
#     print(f"📚 Loaded {len(docs)} chunks from {len(journal_content_map)} journals")

#     vectordb = init_vector_store(docs, PERSIST_DIR)
#     vectordb.persist()

#     qa_chain = init_qa_chain(vectordb, MODEL_NAME)
#     chat_loop(qa_chain, vectordb)


## ========= CHATBOT UI USING GRADIO ===========
# Variabel session state
session_state = {
    "last_search_query": None,
    "last_search_results": [],
    "current_page": 0,
    "page_size": 10,
}

# Fungsi response chatbot
def chatbot_response(user_input, history):
    user_input = user_input.strip()
    response = ""

    # Access global variables
    global qa_chain, vectordb, session_state

    # ===== Command: exit (not needed in Gradio, just for info)
    if user_input.lower() in ("exit", "quit"):
        response = "🔴 Gunakan tombol 'Clear' di Gradio untuk mengulang sesi."
        return response

    elif user_input.lower().startswith("search "):
      query = user_input[7:].strip().lower()
      session_state["last_search_query"] = query
      session_state["current_page"] = 0

      # 1. Ambil hingga 100 chunk teratas berdasar embedding‐similarity
      raw_results = vectordb.similarity_search_with_score(query, k=100)
      raw_results.sort(key=lambda x: x[1], reverse=True)

      # 2. Filter hanya chunk yang secara literal mengandung kata “query”
      filtered = [
          (doc, score)
          for doc, score in raw_results
          if query in doc.page_content.lower()
      ]

      # 3. Jika setelah filtering kosong, fallback ke semantic‐only
      if filtered:
          session_state["last_search_results"] = filtered
      else:
          session_state["last_search_results"] = raw_results

      # 4. Kalau benar‐benar kosong (tidak ada semantic + tidak ada literal), tampilkan pesan
      if not session_state["last_search_results"]:
          return "❌ Tidak ditemukan."

      # 5. Tampilkan page pertama menggunakan display_search_page_text
      response = display_search_page_text(
          session_state["last_search_results"],
          session_state["current_page"],
          session_state["page_size"]
      )
      return response

    # ===== Command: search
    # elif user_input.startswith("search "):
    #     query = user_input[7:].strip()
    #     session_state["last_search_query"] = query
    #     session_state["current_page"] = 0
    #     session_state["last_search_results"] = vectordb.similarity_search_with_score(query, k=100)
    #     session_state["last_search_results"].sort(key=lambda x: x[1], reverse=True)

    #     if not session_state["last_search_results"]:
    #         response = "❌ Tidak ditemukan"
    #     else:
    #         response = display_search_page_text(session_state["last_search_results"], session_state["current_page"], session_state["page_size"])

    # ===== Command: next page
    # elif user_input.lower() == "next":
    #     if not session_state["last_search_query"]:
    #         response = "❌ Belum ada pencarian. Gunakan 'search [topik]' dahulu."
    #     else:
    #         session_state["current_page"] += 1
    #         max_pages = (len(session_state["last_search_results"]) + session_state["page_size"] - 1) // session_state["page_size"]
    #         if session_state["current_page"] >= max_pages:
    #             session_state["current_page"] = max_pages - 1
    #             response = "❌ Tidak ada halaman selanjutnya."
    #         else:
    #             response = display_search_page_text(session_state["last_search_results"], session_state["current_page"], session_state["page_size"])
    elif user_input.lower().startswith("search "):
      query = user_input[7:].strip().lower()
      session_state["last_search_query"] = query
      session_state["current_page"] = 0

      # 1. Ambil hingga 100 chunk teratas berdasar embedding‐similarity
      raw_results = vectordb.similarity_search_with_score(query, k=100)
      raw_results.sort(key=lambda x: x[1], reverse=True)

      # 2. Filter hanya chunk yang secara literal mengandung kata “query”
      filtered = [
          (doc, score)
          for doc, score in raw_results
          if query in doc.page_content.lower()
      ]

      # 3. Jika setelah filtering kosong, fallback ke semantic‐only
      if filtered:
          session_state["last_search_results"] = filtered
      else:
          session_state["last_search_results"] = raw_results

      # 4. Kalau benar‐benar kosong (tidak ada semantic + tidak ada literal), tampilkan pesan
      if not session_state["last_search_results"]:
          return "❌ Tidak ditemukan."

      # 5. Tampilkan page pertama menggunakan display_search_page_text
      response = display_search_page_text(
          session_state["last_search_results"],
          session_state["current_page"],
          session_state["page_size"]
      )
      return response
    elif user_input.lower() == "next":
        if not session_state["last_search_query"]:
            return "❌ Belum ada pencarian. Gunakan 'search [topik]' dahulu."

        session_state["current_page"] += 1
        max_pages = (len(session_state["last_search_results"]) + session_state["page_size"] - 1) // session_state["page_size"]

        if session_state["current_page"] >= max_pages:
            session_state["current_page"] = max_pages - 1
            return "❌ Tidak ada halaman selanjutnya."
        else:
            return display_search_page_text(
                session_state["last_search_results"],
                session_state["current_page"],
                session_state["page_size"]
            )

    # ===== Command: summary
    elif user_input.startswith("summary "):
        title = user_input[8:].strip()
        matches = [f for f in journal_content_map if title.lower() in f.lower()]
        if not matches:
            response = "❌ Tidak ada jurnal dengan judul itu."
        elif len(matches) == 1:
            response = summarize_journal(matches[0])
        else:
            response = "📚 Beberapa kandidat ditemukan:\n"
            for i, f in enumerate(matches):
                response += f"{i+1}. {f}\n"
            response += "\n(🚫 Untuk pemilihan manual, fitur belum didukung di chatbot.)"

    # ===== Pertanyaan biasa (QA)
    else:
      try:
          result = qa_chain(user_input)  # ← jika ini error, akan langsung lompat ke except
          response = result.get('answer', '❌ Tidak ada jawaban.')
          shown = set()
          if result.get("source_documents"):
              for doc in result['source_documents']:
                  source = doc.metadata.get('source', '')
                  if source:
                      base = os.path.splitext(os.path.basename(source))[0]
                      if base not in shown:
                          shown.add(base)
              if shown:
                  response += "\n📚 Referensi:\n" + "\n".join(f"- {r}" for r in shown)
              else:
                  response += "\n📚 Tidak ada referensi ditemukan."
          else:
              response += "\n📚 Tidak ada referensi ditemukan."

      except Exception as e:
        print(f"❌ ERROR: {e}")
        return f"⚠️ Terjadi error saat menjawab: {str(e)}"

    return response


if __name__ == '__main__':
    docs = load_and_clean_pdfs(JOURNAL_FOLDER)
    hitung = sum(1 for d in docs if 'diabetes' in d.page_content.lower())
    print(f"→ Total chunk yang mengandung kata 'diabetes': {hitung}")

    print(f"📚 Loaded {len(docs)} chunks from {len(journal_content_map)} journals")


    vectordb = init_vector_store(docs, PERSIST_DIR)
    vectordb.persist()

    qa_chain = init_qa_chain(vectordb, MODEL_NAME)

    #showing chatbot interface
    gr.ChatInterface(fn=chatbot_response,
                    title="📚 Journal Assistant Chatbot",
                    chatbot=gr.Chatbot(type="messages", height=400),
                    textbox=gr.Textbox(placeholder="Ketik 'search', 'summary', atau pertanyaan biasa..."),
                    theme="default"
                    ).launch(debug=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful